In [2]:
!wget https://bitbucket.org/newsstuff/headlinesdata/raw/3d468c6c414ffb591f6e685644da25aab1621bdf/stock_news.csv

--2024-05-03 21:40:04--  https://bitbucket.org/newsstuff/headlinesdata/raw/3d468c6c414ffb591f6e685644da25aab1621bdf/stock_news.csv
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2227283 (2.1M) [text/plain]
Saving to: ‘stock_news.csv’

stock_news.csv      100%[===================>]   2.12M  1.82MB/s    in 1.2s    

2024-05-03 21:40:05 (1.82 MB/s) - ‘stock_news.csv’ saved [2227283/2227283]



In [4]:
import pandas as pd

df = pd.read_csv('stock_news.csv')

print(df.head())

   Unnamed: 0                                           headline     label
0           0  Markets Close Mostly Lower Again; ROST, PANW R...  Negative
1           1  Gap plummets on earnings miss, cuts full-year ...  Negative
2           2  Billionaire Ken Fisher is Selling These 10 Stocks  Negative
3           3          Corning net income drops 13%, shares fall  Negative
4           4  Internet Explorer shutdown to cause Japan prob...  Negative


In [7]:
print(df.columns)

del df['Unnamed: 0']

Index(['headline', 'label'], dtype='object')


KeyError: 'Unnamed: 0'

In [20]:
good = []
bad = []
neutral = []

for index, row in df.iterrows():
    if row['label'] == 'Positive':
        good.append(row['headline'])
    elif row['label'] == 'Negative':
        bad.append(row['headline'])
    elif row['label'] == 'Neutral':
        neutral.append(row['headline'])

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.svm as svm 

vC = CountVectorizer()

vT = TfidfVectorizer()

clf = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

In [28]:
goodC = vC.fit_transform(good)
badC = vC.fit_transform(bad)
neutralC = vC.fit_transform(neutral)

goodT = vT.fit_transform(good)
badT = vT.fit_transform(bad)
neutralT = vT.fit_transform(neutral)

In [25]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/himme/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [26]:
goodS = []

for headline in good:
    goodS.append(sia.polarity_scores(headline)['compound'])
    
badS = []

for headline in bad:
    badS.append(sia.polarity_scores(headline)['compound'])
    
neutralS = []

for headline in neutral:
    neutralS.append(sia.polarity_scores(headline)['compound'])

In [31]:
import numpy as np

goodFeatures = np.hstack((goodC.toarray(), goodT.toarray(), np.array(goodS).reshape(-1, 1)))

badFeatures = np.hstack((badC.toarray(), badT.toarray(), np.array(badS).reshape(-1, 1)))

neutralFeatures = np.hstack((neutralC.toarray(), neutralT.toarray(), np.array(neutralS).reshape(-1, 1)))

In [ ]:
numGood = len(goodFeatures)
numBad = len(badFeatures)
numNeutral = len(neutralFeatures)

X = np.vstack((goodFeatures, badFeatures, neutralFeatures))